In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#### This notebook provides initial exploratory analysis of the HPSA designations
Info about HPSAs:
    -https://bhw.hrsa.gov/workforce-shortage-areas/shortage-designation
    -https://bhw.hrsa.gov/workforce-shortage-areas/shortage-designation/scoring
    -https://bhw.hrsa.gov/workforce-shortage-areas/shortage-designation/modernization-project

In [2]:
tn_hpsa = pd.read_excel('../capstone_data/HRSA_HPSA_details.xlsx')
tn_hpsa

,HPSA Name,HPSA ID,Designation Type,HPSA Discipline Class,HPSA Score,State,HPSA Status,HPSA Designation Date,HPSA Designation Last Update,Withdrawn Date,...,% of Population Below 100% Poverty,HPSA Formal Ratio,Address,City,ZIP,HHS Region,Rural Status,Longitude,Latitude,County
0,Overton,147133,Geographic HPSA,Primary Care,0,TN,Withdrawn,12/17/1980,01/21/1994,01/21/1994,...,17.9,3172:1,NaN,NaN,NaN,Region 4 ...,Rural,NaN,NaN,"Overton County, TN"
1,"WTPC, LLC",1471038172,Rural Health Clinic,Primary Care,17,TN,Designated,09/30/2021,09/30/2021,NaN,...,NaN,NaN,557 W Park Pl,Henderson,38340-2027,Region 4 ...,Rural,-88.661904,35.442291,"Chester County, TN"
2,Decatur,147039,Geographic HPSA,Primary Care,0,TN,Withdrawn,07/19/1978,09/13/2010,09/13/2010,...,NaN,3651:1,NaN,NaN,NaN,Region 4 ...,Rural,NaN,NaN,"Decatur County, TN"
3,"Health, Tennessee Dept Of",747999477U,Federally Qualified Health Center,Mental Health,19,TN,Designated,08/01/2015,09/11/2021,NaN,...,NaN,NaN,710 James Robertson Pkwy 64 Andrew Johnson Tower,Nashville,37243-0001,Region 4 ...,Non-Rural,-86.786927,36.167605,"Davidson County, TN"
4,TENNOVA URGENT CARE - SHELBYVILLE,747999477F,Rural Health Clinic,Mental Health,15,TN,Withdrawn,10/25/2010,02/06/2020,02/06/2020,...,NaN,NaN,1612 N Main St STE A,Shelbyville,37160-2392,Region 4 ...,Rural,-86.456820,35.506176,"Bedford County, TN"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,LI-Loudon County,1473386147,HPSA Population,Primary Care,12,TN,Designated,03/18/2022,03/18/2022,NaN,...,11.2,4163:1,NaN,NaN,NaN,Region 4 ...,Non-Rural,NaN,NaN,"Loudon County, TN"
607,LI-Putnam/White Counties,1472793573,HPSA Population,Primary Care,16,TN,Proposed For Withdrawal,03/22/2012,09/10/2021,NaN,...,24.3,5114:1,NaN,NaN,NaN,Region 4 ...,Rural,NaN,NaN,"Putnam County, TN | White County, TN"
608,LI-McNairy County,1472279043,HPSA Population,Primary Care,17,TN,Designated,04/05/2022,04/05/2022,NaN,...,18.8,17452:1,NaN,NaN,NaN,Region 4 ...,Rural,NaN,NaN,"McNairy County, TN"
609,LI-Sevier County,1471717306,HPSA Population,Primary Care,14,TN,Designated,02/28/2014,09/10/2021,NaN,...,13.6,6807:1,NaN,NaN,NaN,Region 4 ...,Rural,NaN,NaN,"Sevier County, TN"


In [3]:
#dropping columns that aren't needed for this analysis
tn_hpsa = tn_hpsa.drop(columns = ['HPSA Degree of Shortage', 'HHS Region'])

#reorganizing columns to allow most pertinent to be readily visible in jupyter notebook
tn_hpsa = tn_hpsa[['HPSA Name', 'HPSA ID', 'Designation Type', 'HPSA Discipline Class', 'HPSA Population Type',
                   'HPSA Score', 'HPSA Status', 'HPSA Designation Date', 'HPSA Designation Last Update', 'Withdrawn Date', 
                   'Longitude', 'Latitude','Address', 'ZIP', 'City', 'State', 'County', 
                   'HPSA FTE', '# of FTE Short', 'HPSA Formal Ratio', 'HPSA Designation Population', 
                   '% of Population Below 100% Poverty', 'Metropolitan Indicator', 'Rural Status']]

#filling in null values in the County column
tn_hpsa.loc[tn_hpsa['County'].isna(), ['County']] = 'Not Entered'

#creating a column to identify those rows that contain HPSA designations that cover multiple counties
tn_hpsa.loc[tn_hpsa['County'].str.contains(pat='\|'), ['Multi-County']] = 'Yes'
tn_hpsa.loc[~tn_hpsa['County'].str.contains(pat='\|'), ['Multi-County']] = 'No'

#Splitting out the multi-county designations so that each county is represented singley in its own row 
tn_hpsa['County'] = tn_hpsa['County'].str.split('|')
tn_hpsa = tn_hpsa.apply(pd.Series.explode)
tn_hpsa = tn_hpsa.reset_index(drop=True)

tn_hpsa

,HPSA Name,HPSA ID,Designation Type,HPSA Discipline Class,HPSA Population Type,HPSA Score,HPSA Status,HPSA Designation Date,HPSA Designation Last Update,Withdrawn Date,...,State,County,HPSA FTE,# of FTE Short,HPSA Formal Ratio,HPSA Designation Population,% of Population Below 100% Poverty,Metropolitan Indicator,Rural Status,Multi-County
0,Overton,147133,Geographic HPSA,Primary Care,Geographic Population,0,Withdrawn,12/17/1980,01/21/1994,01/21/1994,...,TN,"Overton County, TN",5.5000,NaN,3172:1,17444.0,17.9,Non-Metropolitan,Rural,No
1,"WTPC, LLC",1471038172,Rural Health Clinic,Primary Care,NaN,17,Designated,09/30/2021,09/30/2021,NaN,...,TN,"Chester County, TN",NaN,NaN,NaN,59090.0,NaN,Unknown,Rural,No
2,Decatur,147039,Geographic HPSA,Primary Care,Geographic Population,0,Withdrawn,07/19/1978,09/13/2010,09/13/2010,...,TN,"Decatur County, TN",3.1000,0.0000,3651:1,11318.0,NaN,Non-Metropolitan,Rural,No
3,"Health, Tennessee Dept Of",747999477U,Federally Qualified Health Center,Mental Health,NaN,19,Designated,08/01/2015,09/11/2021,NaN,...,TN,"Davidson County, TN",NaN,NaN,NaN,17229.0,NaN,Unknown,Non-Rural,No
4,TENNOVA URGENT CARE - SHELBYVILLE,747999477F,Rural Health Clinic,Mental Health,NaN,15,Withdrawn,10/25/2010,02/06/2020,02/06/2020,...,TN,"Bedford County, TN",NaN,NaN,NaN,145826.0,NaN,Unknown,Rural,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,LI-Putnam/White Counties,1472793573,HPSA Population,Primary Care,Low Income Population HPSA,16,Proposed For Withdrawal,03/22/2012,09/10/2021,NaN,...,TN,"White County, TN",9.0721,6.3879,5114:1,46395.0,24.3,Unknown,Rural,Yes
737,LI-McNairy County,1472279043,HPSA Population,Primary Care,Low Income Population HPSA,17,Designated,04/05/2022,04/05/2022,NaN,...,TN,"McNairy County, TN",0.6700,3.2300,17452:1,11693.0,18.8,Unknown,Rural,No
738,LI-Sevier County,1471717306,HPSA Population,Primary Care,Low Income Population HPSA,14,Designated,02/28/2014,09/10/2021,NaN,...,TN,"Sevier County, TN",5.4700,6.9400,6807:1,37235.0,13.6,Unknown,Rural,No
739,LI-Macon &amp; Trousdale Counties,1471425309,HPSA Population,Primary Care,Low Income Population HPSA,16,Designated,04/05/2022,04/05/2022,NaN,...,TN,"Macon County, TN",1.3900,2.6700,8755:1,12169.0,15.2,Unknown,Rural,Yes


In [57]:
#the duplicated statement is accounting for those multi-county HPSAs that I exploded out into multiple rows
tn_hpsa.loc[~tn_hpsa.duplicated(['HPSA ID'])]['Rural Status'].value_counts()

Rural              386
Non-Rural          172
Partially Rural     39
Unknown             14
Name: Rural Status, dtype: int64

In [58]:
tn_hpsa.loc[~tn_hpsa.duplicated(['HPSA ID'])]['Rural Status'].value_counts(normalize=True)

Rural              0.631751
Non-Rural          0.281506
Partially Rural    0.063830
Unknown            0.022913
Name: Rural Status, dtype: float64

In [59]:
tn_hpsa.loc[~tn_hpsa.duplicated(['HPSA ID'])]['HPSA Discipline Class'].value_counts()

Primary Care     285
Dental Health    211
Mental Health    115
Name: HPSA Discipline Class, dtype: int64

In [60]:
tn_hpsa.loc[~tn_hpsa.duplicated(['HPSA ID'])]['HPSA Discipline Class'].value_counts(normalize=True)

Primary Care     0.466448
Dental Health    0.345336
Mental Health    0.188216
Name: HPSA Discipline Class, dtype: float64

In [79]:
tn_hpsa.loc[~tn_hpsa.duplicated(['HPSA ID'])]['Designation Type'].value_counts()

HPSA Population                                  250
Rural Health Clinic                              108
Federally Qualified Health Center                 93
Geographic HPSA                                   74
High Needs Geographic HPSA                        44
Correctional Facility                             21
Other Facility                                     8
Federally Qualified Health Center Look A Like      7
State Mental Hospital                              6
Name: Designation Type, dtype: int64

In [80]:
tn_hpsa.loc[~tn_hpsa.duplicated(['HPSA ID'])]['Designation Type'].value_counts(normalize=True)

HPSA Population                                  0.409165
Rural Health Clinic                              0.176759
Federally Qualified Health Center                0.152209
Geographic HPSA                                  0.121113
High Needs Geographic HPSA                       0.072013
Correctional Facility                            0.034370
Other Facility                                   0.013093
Federally Qualified Health Center Look A Like    0.011457
State Mental Hospital                            0.009820
Name: Designation Type, dtype: float64

### The next section hones in on those HPSA designations that are currently active and apply to rural and partially-rural areas

In [4]:
tn_rur_hpsa = tn_hpsa.loc[(tn_hpsa['HPSA Status'] != 'Withdrawn')
                         & (tn_hpsa['Rural Status'] != 'Non-Rural')].reset_index(drop=True)
tn_rur_hpsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   HPSA Name                           351 non-null    object 
 1   HPSA ID                             351 non-null    object 
 2   Designation Type                    351 non-null    object 
 3   HPSA Discipline Class               351 non-null    object 
 4   HPSA Population Type                271 non-null    object 
 5   HPSA Score                          351 non-null    int64  
 6   HPSA Status                         351 non-null    object 
 7   HPSA Designation Date               351 non-null    object 
 8   HPSA Designation Last Update        351 non-null    object 
 9   Withdrawn Date                      0 non-null      object 
 10  Longitude                           80 non-null     float64
 11  Latitude                            80 non-nu

In [71]:
tn_rur_hpsa.loc[~tn_rur_hpsa.duplicated(['HPSA ID'])]['Rural Status'].value_counts()

Rural              226
Partially Rural     26
Name: Rural Status, dtype: int64

In [71]:
tn_rur_hpsa.loc[~tn_rur_hpsa.duplicated(['HPSA ID'])]['Rural Status'].value_counts(normalize=True)

Rural              226
Partially Rural     26
Name: Rural Status, dtype: int64

In [72]:
tn_rur_hpsa.loc[~tn_rur_hpsa.duplicated(['HPSA ID'])]['Designation Type'].value_counts()

HPSA Population                      135
Federally Qualified Health Center     36
Rural Health Clinic                   33
Geographic HPSA                       19
High Needs Geographic HPSA            18
Correctional Facility                 10
State Mental Hospital                  1
Name: Designation Type, dtype: int64

In [81]:
tn_rur_hpsa.loc[~tn_rur_hpsa.duplicated(['HPSA ID'])]['Designation Type'].value_counts(normalize=True)

HPSA Population                      0.535714
Federally Qualified Health Center    0.142857
Rural Health Clinic                  0.130952
Geographic HPSA                      0.075397
High Needs Geographic HPSA           0.071429
Correctional Facility                0.039683
State Mental Hospital                0.003968
Name: Designation Type, dtype: float64

In [73]:
tn_rur_hpsa.loc[~tn_rur_hpsa.duplicated(['HPSA ID'])]['HPSA Status'].value_counts()

Designated                 218
Proposed For Withdrawal     34
Name: HPSA Status, dtype: int64

In [74]:
tn_rur_hpsa.loc[~tn_rur_hpsa.duplicated(['HPSA ID'])]['HPSA Discipline Class'].value_counts()

Primary Care     104
Dental Health    104
Mental Health     44
Name: HPSA Discipline Class, dtype: int64

In [75]:
tn_rur_hpsa.loc[~tn_rur_hpsa.duplicated(['HPSA ID'])]['HPSA Discipline Class'].value_counts(normalize=True)

Primary Care     0.412698
Dental Health    0.412698
Mental Health    0.174603
Name: HPSA Discipline Class, dtype: float64

In [8]:
#splitting out the different types of HPSA designations (facility, geographic, population)
tn_rur_hpsa_fac = tn_rur_hpsa.loc[tn_rur_hpsa['Designation Type'].isin(['Federally Qualified Health Center', 
                                                                       'Rural Health Clinic', 
                                                                        'Federally Qualified Health Center Look A Like'])]
tn_rur_hpsa_geo = tn_rur_hpsa.loc[tn_rur_hpsa['Designation Type'].isin(['Geographic HPSA', 
                                                                       'High Needs Geographic HPSA'])]
tn_rur_hpsa_pop = tn_rur_hpsa.loc[tn_rur_hpsa['Designation Type'] =='HPSA Population']

#### Facility-based HPSA info

In [77]:
tn_rur_hpsa_fac.loc[~tn_rur_hpsa_fac.duplicated(['HPSA ID'])]['HPSA Discipline Class'].value_counts()

Primary Care     23
Mental Health    23
Dental Health    23
Name: HPSA Discipline Class, dtype: int64

In [78]:
tn_rur_hpsa_fac.loc[~tn_rur_hpsa_fac.duplicated(['HPSA ID'])]['HPSA Discipline Class'].value_counts(normalize=True)

Primary Care     0.333333
Mental Health    0.333333
Dental Health    0.333333
Name: HPSA Discipline Class, dtype: float64

In [11]:
tn_rur_hpsa_fac[['County', 'HPSA Discipline Class']].value_counts()

County                 HPSA Discipline Class
Campbell County, TN    Dental Health            3
                       Mental Health            3
                       Primary Care             3
Hardeman County, TN    Mental Health            2
                       Dental Health            2
                       Primary Care             2
Bedford County, TN     Dental Health            1
Meigs County, TN       Mental Health            1
                       Dental Health            1
Maury County, TN       Primary Care             1
                       Mental Health            1
                       Dental Health            1
Monroe County, TN      Dental Health            1
Lake County, TN        Primary Care             1
                       Mental Health            1
                       Dental Health            1
Johnson County, TN     Primary Care             1
Meigs County, TN       Primary Care             1
Monroe County, TN      Primary Care             1
     

In [17]:
tn_rur_hpsa_fac[['HPSA Score', 'County']].value_counts().sort_index(ascending=False)

HPSA Score  County               
25          Polk County, TN          1
            Hawkins County, TN       1
            Hardin County, TN        1
            Cocke County, TN         1
            Campbell County, TN      2
23          Lake County, TN          1
            Hardin County, TN        1
22          Cocke County, TN         1
21          Scott County, TN         1
            Perry County, TN         1
            Lake County, TN          1
            Hawkins County, TN       1
            Hardeman County, TN      1
            Campbell County, TN      2
20          Scott County, TN         2
            Perry County, TN         1
            Monroe County, TN        1
            Hardeman County, TN      1
            Campbell County, TN      2
19          Polk County, TN          2
            Perry County, TN         1
            Monroe County, TN        1
            Maury County, TN         1
            Lake County, TN          1
            Johnson County, TN

In [82]:
tn_rur_hpsa_fac.loc[~tn_rur_hpsa_fac.duplicated(['HPSA ID'])][['HPSA Discipline Class', 
                                                               'HPSA Score']].value_counts().sort_index(ascending=False)

HPSA Discipline Class  HPSA Score
Primary Care           21            1
                       20            3
                       19            4
                       18            2
                       17            4
                       16            1
                       15            4
                       14            2
                       11            1
                       8             1
Mental Health          23            1
                       22            1
                       21            3
                       20            4
                       19            2
                       18            4
                       17            6
                       15            2
Dental Health          25            6
                       23            1
                       21            3
                       19            4
                       17            4
                       15            3
                       13     

#### Geographically-based HPSA info

In [83]:
tn_rur_hpsa_geo.loc[~tn_rur_hpsa_geo.duplicated(['HPSA ID'])]['HPSA Discipline Class'].value_counts()

Primary Care     24
Mental Health     9
Dental Health     4
Name: HPSA Discipline Class, dtype: int64

In [29]:
tn_rur_hpsa_geo[['County']].value_counts()

County                
Wayne County, TN          2
Lake County, TN           2
Grundy County, TN         2
Hancock County, TN        2
Carter County, TN         1
                         ..
 Robertson County, TN     1
 Stewart County, TN       1
 Sullivan County, TN      1
 Unicoi County, TN        1
 Weakley County, TN       1
Length: 68, dtype: int64

In [84]:
tn_rur_hpsa_geo.loc[~tn_rur_hpsa_geo.duplicated(['HPSA ID'])][['HPSA Score']].value_counts().sort_index(ascending=False)

HPSA Score
20            1
19            1
18            3
17            3
16            3
15            4
14            2
13            6
12            5
11            5
10            2
9             2
dtype: int64

In [85]:
tn_rur_hpsa_geo.loc[~tn_rur_hpsa_geo.duplicated(['HPSA ID'])][['HPSA Discipline Class', 
                                                               'HPSA Score']].value_counts().sort_index(ascending=False)

HPSA Discipline Class  HPSA Score
Primary Care           19            1
                       18            2
                       17            2
                       16            2
                       15            2
                       13            4
                       12            3
                       11            5
                       10            1
                       9             2
Mental Health          18            1
                       17            1
                       16            1
                       15            1
                       14            1
                       13            2
                       12            2
Dental Health          20            1
                       15            1
                       14            1
                       10            1
dtype: int64

#### Population-based HPSA info

In [86]:
tn_rur_hpsa_pop.loc[~tn_rur_hpsa_pop.duplicated(['HPSA ID'])]['HPSA Discipline Class'].value_counts()

Dental Health    73
Primary Care     51
Mental Health    11
Name: HPSA Discipline Class, dtype: int64

In [34]:
tn_rur_hpsa_pop[['County', 'HPSA Discipline Class']].value_counts()

County                 HPSA Discipline Class
Putnam County, TN      Primary Care             2
 White County, TN      Primary Care             2
 Campbell County, TN   Mental Health            1
Humphreys County, TN   Dental Health            1
Haywood County, TN     Primary Care             1
                                               ..
Bledsoe County, TN     Dental Health            1
Bledsoe County, TN     Mental Health            1
Blount County, TN      Dental Health            1
                       Primary Care             1
White County, TN       Dental Health            1
Length: 197, dtype: int64

In [87]:
tn_rur_hpsa_pop.loc[~tn_rur_hpsa_pop.duplicated(['HPSA ID'])]['HPSA Population Type'].value_counts()

Low Income Population HPSA    135
Name: HPSA Population Type, dtype: int64

In [88]:
tn_rur_hpsa_pop.loc[~tn_rur_hpsa_pop.duplicated(['HPSA ID'])][['HPSA Score']].value_counts().sort_index(ascending=False)

HPSA Score
19             8
18            14
17            37
16            16
15            24
14             5
13            12
12             5
11             5
10             1
9              3
8              1
7              4
dtype: int64

In [93]:
tn_rur_hpsa_pop.loc[~tn_rur_hpsa_pop.duplicated(['HPSA ID'])][['HPSA Discipline Class', 
                                                               'HPSA Score']].value_counts().sort_index(ascending=False)

HPSA Discipline Class  HPSA Score
Primary Care           18             4
                       17             6
                       16            10
                       15            10
                       14             3
                       13             6
                       12             4
                       11             2
                       10             1
                       9              2
                       8              1
                       7              2
Mental Health          17             8
                       16             2
                       15             1
Dental Health          19             8
                       18            10
                       17            23
                       16             4
                       15            13
                       14             2
                       13             6
                       12             1
                       11             3
      

#### Exploring HPSA designations that have been proposed for withdrawal

In [39]:
tn_hpsa_prop_wi = tn_hpsa.loc[tn_hpsa['HPSA Status'] == 'Proposed For Withdrawal']

In [94]:
tn_hpsa_prop_wi.loc[~tn_hpsa_prop_wi.duplicated(['HPSA ID'])]['Designation Type'].value_counts()

HPSA Population               15
Correctional Facility         10
Geographic HPSA                7
High Needs Geographic HPSA     6
State Mental Hospital          3
Name: Designation Type, dtype: int64

In [95]:
tn_hpsa_prop_wi.loc[~tn_hpsa_prop_wi.duplicated(['HPSA ID'])]['Designation Type'].value_counts(normalize=True)

HPSA Population               0.365854
Correctional Facility         0.243902
Geographic HPSA               0.170732
High Needs Geographic HPSA    0.146341
State Mental Hospital         0.073171
Name: Designation Type, dtype: float64

In [96]:
tn_hpsa_prop_wi.loc[~tn_hpsa_prop_wi.duplicated(['HPSA ID'])]['Rural Status'].value_counts()

Rural              26
Partially Rural     8
Non-Rural           7
Name: Rural Status, dtype: int64

In [97]:
tn_hpsa_prop_wi.loc[~tn_hpsa_prop_wi.duplicated(['HPSA ID'])]['Rural Status'].value_counts(normalize=True)

Rural              0.634146
Partially Rural    0.195122
Non-Rural          0.170732
Name: Rural Status, dtype: float64

In [100]:
tn_rur_hpsa_prop_wi_fac = tn_hpsa_prop_wi.loc[(tn_hpsa_prop_wi['Designation Type'].isin(['State Mental Hospital']))
                                     & (tn_hpsa_prop_wi['Rural Status'] != 'Non-Rural')]

tn_rur_hpsa_prop_wi_geo = tn_hpsa_prop_wi.loc[(tn_hpsa_prop_wi['Designation Type'].isin(['Geographic HPSA', 
                                                                       'High Needs Geographic HPSA']))
                                     & (tn_hpsa_prop_wi['Rural Status'] != 'Non-Rural')]

tn_rur_hpsa_prop_wi_pop = tn_hpsa_prop_wi.loc[tn_hpsa_prop_wi['Designation Type'].isin(['HPSA Population'])
                                 & (tn_hpsa_prop_wi['Rural Status'] != 'Non-Rural')]

In [104]:
tn_rur_hpsa_prop_wi_fac

,HPSA Name,HPSA ID,Designation Type,HPSA Discipline Class,HPSA Population Type,HPSA Score,HPSA Status,HPSA Designation Date,HPSA Designation Last Update,Withdrawn Date,...,State,County,HPSA FTE,# of FTE Short,HPSA Formal Ratio,HPSA Designation Population,% of Population Below 100% Poverty,Metropolitan Indicator,Rural Status,Multi-County
218,Western Mental Health Institute,7474011436,State Mental Hospital,Mental Health,NaN,12,Proposed For Withdrawal,08/28/2017,03/30/2022,NaN,...,TN,"Hardeman County, TN",3.25,NaN,NaN,141.0,NaN,Unknown,Rural,No


In [105]:
tn_rur_hpsa_prop_wi_geo[~tn_rur_hpsa_prop_wi_geo.duplicated(['HPSA ID'])]['HPSA Discipline Class'].value_counts()

Primary Care     8
Mental Health    4
Name: HPSA Discipline Class, dtype: int64

In [106]:
tn_rur_hpsa_prop_wi_pop[~tn_rur_hpsa_prop_wi_pop.duplicated(['HPSA ID'])]['HPSA Discipline Class'].value_counts()

Dental Health    7
Primary Care     5
Name: HPSA Discipline Class, dtype: int64

In [107]:
tn_rur_hpsa_prop_wi_pop['County'].value_counts()

Robertson County, TN      2
Blount County, TN         2
Maury County, TN          2
Cheatham County, TN       2
Trousdale County, TN      1
Rutherford County, TN     1
 Wilson County, TN        1
Moore County, TN          1
Putnam County, TN         1
 White County, TN         1
Name: County, dtype: int64

In [108]:
tn_rur_hpsa_prop_wi_pop[~tn_rur_hpsa_prop_wi_pop.duplicated(['HPSA ID'])]['HPSA Population Type'].value_counts()

Low Income Population HPSA    12
Name: HPSA Population Type, dtype: int64